In [62]:
import pickle
from PIL import Image
import numpy as np
import gradio as gr
from tensorflow.keras.models import load_model
import tensorflow as tf
from pathlib import Path
import openai
import os
from dotenv import load_dotenv
from transformers import pipeline


In [63]:
# Load environment variables from the .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OpenAI API Key Loaded: {openai_api_key is not None}")

# Set the OpenAI API key
openai.api_key = openai_api_key


OpenAI API Key Loaded: True


In [64]:
# Set the model's file path
file_path = Path("models/model_adam_scaled.h5")
print(f"Loading model from {file_path}")

# Load the model to a new object
adam_scaled = tf.keras.models.load_model(file_path)

# Compile the model
adam_scaled.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)
print("Model compiled successfully.")


Loading model from models/model_adam_scaled.h5


Model compiled successfully.


In [65]:
# Define a function to generate explanations
def generate_explanation(prediction):
    prompt = (
        f"The model has predicted the image as {prediction}. "
        "Explain what this means in simple terms and provide any relevant information."
    )
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Use the ChatGPT-4 model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=0.7,
    )
    explanation = response['choices'][0]['message']['content'].strip()
    return explanation

# Define the Gradio prediction function
def predict_image(img):
    try:
        print("Resizing image...")
        img = img.resize((224, 224))
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        print(f"Image shape: {img_array.shape}")
        prediction = adam_scaled.predict(img_array)
        print(f"Prediction: {prediction[0][0]}")
        return "Malignant" if prediction[0][0] > 0.5 else "Benign"
    except Exception as e:
        print(f'Error: {e}')
        return 'There was an error processing the image. Please try again.'


In [66]:
# Define a new function to get prediction and explanation
def predict_and_explain(img):
    # Get the prediction
    prediction_label = predict_image(img)
    
    # Generate an explanation
    explanation = generate_explanation(prediction_label)
    
    # Combine the prediction and explanation
    result = f"Prediction: {prediction_label}\n\nExplanation: {explanation}"
    return result


In [67]:
# Create Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(image_mode='RGB', type='pil'),
    outputs="text",
    title="Skin Cancer Detection Model",
    description="Upload an image, and the model will predict whether it's malignant or benign, with an explanation."
)

# Launch the interface
interface.launch()

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


Resizing image...
Image shape: (1, 224, 224, 3)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction: 0.17953187227249146
Resizing image...
Image shape: (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Prediction: 0.9478165507316589
Resizing image...
Image shape: (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Prediction: 0.9478165507316589
